In [1]:
using BenchmarkTools: @btime

In [96]:
include("basis.jl")

trapz (generic function with 1 method)

In [110]:
a = 0.01

n = 1000
l = 12
grid = rand(n);
spfs = [rand(n) for i in 1:l];
#spfs = [rand(n) .+ rand(n)im for i in 1:l];

In [111]:
inner  = inner_ints(spfs, grid, a);

In [125]:
function outer_int2(spfs, grid, inner_ints)
    l = length(spfs)
    outer_int = zeros(typeof(spfs[1][1]), l, l, l, l)
    
    fs = [similar(spfs[1]) for i in 1:Threads.nthreads()]
    is = [similar(spfs[1]) for i in 1:Threads.nthreads()]

    @inbounds Threads.@threads for κ in 1:l
        f_vals = fs[Threads.threadid()]
        inner = is[Threads.threadid()]
        
        for λ in 1:l
            @views inner .= inner_ints[κ, λ, :]
            for μ in 1:l
                for ν in μ:l
                    f_vals .= conj.(spfs[μ]) .* inner .* spfs[ν]
                    outer_int[μ, κ, ν, λ] = trapz(f_vals, grid)
                    outer_int[ν, λ, μ, κ] = outer_int[μ, κ, ν, λ]'
                end
            end
        end
    end
    return outer_int
end

outer_int2 (generic function with 1 method)

In [126]:
@time outer = outer_int(spfs, grid, inner);

  0.003214 seconds (63 allocations: 294.266 KiB)


In [127]:
@time outer = outer_int2(spfs, grid, inner);

  0.266912 seconds (277.15 k allocations: 14.327 MiB, 99.23% compilation time)


In [83]:
outer_int(spfs, grid, inner) ≈ outer_int2(spfs, grid, inner)

true

In [19]:
i = 1
j = 4
k = 2
l = 6

outer[i, j, k, l] ≈ outer[k, l, i, j]'

true